In [1]:
import pandas as pd
import requests
import os 
from dotenv import load_dotenv
import requests
from tqdm import tqdm

In [2]:
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials ## para poner mis credenciales

# lastfm
Voy a consultar todos los artistas top de la página de lastfm y voy a sacar los top géneros de dichos artistas. Para ello, se obtienen los datos con los métodos `'chart.gettopartists'` y `'artist.getTopTags'`

Aquí está la docu, para más información: https://www.last.fm/api#getting-started

En primer lugar, voy a introducir mis credenciales como parámetros de entorno.

In [3]:
load_dotenv()

True

In [4]:
api = os.getenv("lastfm-id")
username = os.getenv("lastfm-user")

Para hacer la consulta a la api, tengo que darle como input una serie de parámetros:
- el link al que quiero hacer la petición ( https://ws.audioscrobbler.com/2.0/ )
- `headers`: mis credenciales de usuario de la api / lastfm
- `params`: los parámetros que tengo que meter para poder hacer la consulta a la api. En este caso, serán los siguientes:
  - `api_key`: la clave con la que voy a acceder
  - `method`: los datos a los que quiero acceder. En este caso, `'chart.gettopartists'` para que me de los top artistas de la plataforma.
  - `format`: el formato que quiero que me de de respuesta, pudiendo ser `json` o `xml`

In [5]:
headers = {
    'user-agent': username
}

params = {
    'api_key': api,
    'method': 'chart.gettopartists',
    'format': 'json'
}

res = requests.get('https://ws.audioscrobbler.com/2.0/', headers=headers, params=params)
res.status_code ## si 200 -> éxito

200

In [11]:
res.json()['artists']['artist']

[{'name': 'The Weeknd',
  'playcount': '382431882',
  'listeners': '2954448',
  'mbid': 'c8b03190-306c-4120-bb0b-6f2ebfc06ea9',
  'url': 'https://www.last.fm/music/The+Weeknd',
  'streamable': '0',
  'image': [{'#text': 'https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'small'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'medium'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'large'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'extralarge'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'mega'}]},
 {'name': 'Taylor Swift',
  'playcount': '1062549938',
  'listeners': '3677874',
  'mbid': '20244d07-534f-4eff-b4d4-930878889970',
  'url': 'https://www.last.fm/music/Taylor+Swift',
  'streama

me creo una función para fijar los parámetros fijos, y que el `method` entre como variable.

In [ ]:
## me voy a hacer una función con los datos de la petición
def getLastFMData(params):
    headers = {'user-agent': username} ## defino mi usuario
    url = 'https://ws.audioscrobbler.com/2.0/' ##url a la que voy a contactar

    ## configuro los params para hacer la petición
    params['api_key'] = api
    params['format'] = 'json'

    res = requests.get(url, headers=headers, params=params)
    return res

In [ ]:
## voy a probar que la petición funciona
topArtists = getLastFMData({
    'method': 'chart.gettopartists'
})
topArtists.status_code

In [ ]:
topArtists.json() ## me da un cacharro de tipo json, al que ya me enfrentaré luego

para obtener la info concreta de los artistas, necesito saber la página de la que quiero obtener la info y el número de resultados que quiero obtener por página. https://www.last.fm/api/show/chart.getTopArtists

el número de páginas no sé cuanto es, pero el límite de artistas por páginas es 50

In [ ]:
int(topArtists.json()['artists']['@attr']['totalPages'])

In [ ]:
## voy a intentar obtener la info de los artistas por cada una de las páginas
tqdm.pandas()

responses = [] ## almacenaré cada una de las peticiones por página en una lista

page_inicio = 1
page_final = int(topArtists.json()['artists']['@attr']['totalPages']) #sé el número máx de páginas de la consulta que hice anteriormente
page_limit = int(topArtists.json()['artists']['@attr']['perPage']) #sé el número de artistas que tengo por página

for page in tqdm(range(page_inicio, page_final)):
    if page <= page_final:
        ## cambio los parámetros de entrada 
        params = {
            'method': 'chart.gettopartists',
            'limit': page_limit, 
            'page': page
        }
        
        ## hago la petición con los nuevos parámetros
        res = getLastFMData(params)

        ## puedo sacar la página por la que voy
        page = int(res.json()['artists']['@attr']['page'])

        ## añado las respuesta a la lista
        responses.append(res)

        ## le sumo uno a la página para que avance
        page += 1

voy a juntar los artistas en un dataframe

In [ ]:
## voy a fijarme en la estructura de una de las respuestas que he appendeado a responses
## al final, es un diccionario que tiene dentro otro diccionario y da los valores en listas
responses[0].json()

In [ ]:
responses[0].json()['artists']['artist'] ## así puedo obtener la información como una lista de listas, por la que puedo iterar y convertir en dataframes

In [ ]:
pd.DataFrame(responses[0].json()['artists']['artist'])

In [ ]:
pags = [pd.DataFrame(response.json()['artists']['artist']) for response in responses]
artists = pd.concat(pags)
print(artists.shape)
artists.head()

In [ ]:
artists.info()
artists.describe().T

In [ ]:
artists.to_csv('artists.csv')

extraigo ahora los top géneros de cada artista con `'artist.getTopTags'`

In [ ]:
def getTags(artist):
    ## hago la colsulta para obtener los datos de tags por artista
    res = getLastFMData({
        'method': 'artist.getTopTags',
        'artist':  artist
    })

    # por si me da un timeout
    if res.status_code != 200:
        return None

    # extraigo los top 5 tags de cada artistas
    tags = [tag['name'] for tag in res.json()['toptags']['tag'][:5]]
    tags_str = ', '.join(tags)

In [ ]:
tqdm.pandas()

## esto es un apply pero le pongo la barra chula para que me dé el status
artists['tags'] = artists['name'].progress_apply(getTags)

In [ ]:
artists.head()

In [ ]:
artists.to_csv('artists_with_tags.csv')